# VCF QC and Filtering

## Once we have a VCF file, filtering is still required to get quality data: 

## vcf_qual_filter_v1.py  script

In [2]:
#!/usr/bin/env python3

# Create a script to evaluate quality of vcf file produced by our fastq -> vcf pipeline
# Generate the numbers required to create histograms for read depth and allelic balance from a single VCF
# Generate two lists (variant allele frequencies and variant read depth) for unfiltered VCF, then filter and generate the two lists again for each of the following parameters:
# Will create 10 lists total.

# Filtering parameters:
# 1) GQ >= 30
# 2) AD1 >= 3 AND AD2 >=3
# 3) DP <= 100
# 4) GQ >= 30 AND AD1 >= 3 AND AD2 >=3 AND DP <= 100

import sys
import os

def gen_QC_lists(fh):
    AFs_list = []
    DPs_list = []
    GQ_filtered_AFs_list = []
    GQ_filtered_DPs_list = []
    AD_filtered_AFs_list = []
    AD_filtered_DPs_list = []
    DP_filtered_AFs_list = []
    DP_filtered_DPs_list = []
    threeX_filtered_AFs_list = []
    threeX_filtered_DPs_list = []
    QC_list_of_lists = []
    for line in fh:
        if not line.startswith('#'):
            line = line.rstrip()
            field10 = line.split("\t")[9]
            elements_in_field10 = len(field10.split(":"))
            if elements_in_field10 == 5:
                AD = field10.split(':')[1]
                DP = int(field10.split(':')[2])
                GQ = int(field10.split(':')[3])
                AD1 = float(AD.split(',')[0])
                AD2 = float(AD.split(',')[1])
                if AD1+AD2 != 0:
                    variant_AF = (AD1/(AD1+AD2))
                    AFs_list.append(variant_AF)
                    DPs_list.append(DP)
                    if GQ >= 30:
                        GQ_filtered_AFs_list.append(variant_AF)
                    if GQ >= 30:
                        GQ_filtered_DPs_list.append(DP)
                    if AD1 >= 3 and AD2 >= 3:
                        AD_filtered_AFs_list.append(variant_AF)
                    if AD1 >= 3 and AD2 >= 3:
                        AD_filtered_DPs_list.append(DP)
                    if DP <= 100:
                        DP_filtered_AFs_list.append(variant_AF)
                    if DP <= 100:
                        DP_filtered_DPs_list.append(DP)
                    if GQ >= 30 and AD1 >= 3 and AD2 >= 3 and DP <= 100:
                        threeX_filtered_AFs_list.append(variant_AF)
                    if GQ >= 30 and AD1 >= 3 and AD2 >= 3 and DP <= 100:
                        threeX_filtered_DPs_list.append(DP)
    QC_list_of_lists.append(AFs_list)
    QC_list_of_lists.append(DPs_list)
    QC_list_of_lists.append(GQ_filtered_AFs_list)
    QC_list_of_lists.append(GQ_filtered_DPs_list)
    QC_list_of_lists.append(AD_filtered_AFs_list)
    QC_list_of_lists.append(AD_filtered_DPs_list)
    QC_list_of_lists.append(DP_filtered_AFs_list)
    QC_list_of_lists.append(DP_filtered_DPs_list)
    QC_list_of_lists.append(threeX_filtered_AFs_list)
    QC_list_of_lists.append(threeX_filtered_DPs_list)
    return(QC_list_of_lists)

def gen_filtered_vcf(fn, fh):
    sample_name = (os.path.basename(fn)).split(".")[0]
    filtered_vcf = open(sample_name+".filtered.vcf", "w")
    for line in fh:
        if line.startswith('#'):
            filtered_vcf.write(line)
        else:
            line = line.rstrip()
            field10 = line.split("\t")[9]
            elements_in_field10 = len(field10.split(":"))
            if elements_in_field10 == 5:
                AD = field10.split(':')[1]
                DP = int(field10.split(':')[2])
                GQ = int(field10.split(':')[3])
                AD1 = float(AD.split(',')[0])
                AD2 = float(AD.split(',')[1])
                if GQ >= 30 and AD1 >= 3 and AD2 >= 3 and DP <= 100:
                    filtered_vcf.write(line+"\n")
    return filtered_vcf
    

input_vcf = sys.argv[1]
fh = open(input_vcf, "r")

QC = gen_QC_lists(fh)
for list in QC:
    print(len(list))

fh.seek(0)    
gen_filtered_vcf(input_vcf,fh)    

fh.close()


FileNotFoundError: [Errno 2] No such file or directory: '-f'

### Excerpt of VCF file

In [ ]:
##fileformat=VCFv4.2
##FILTER=<ID=LowQual,Description="Low quality">
##FORMAT=<ID=AD,Number=R,Type=Integer,Description="Allelic depths for the ref and alt alleles in the order listed">
##FORMAT=<ID=DP,Number=1,Type=Integer,Description="Approximate read depth (reads with MQ=255 or with bad mates are filtered)">
##FORMAT=<ID=GQ,Number=1,Type=Integer,Description="Genotype Quality">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=PL,Number=G,Type=Integer,Description="Normalized, Phred-scaled likelihoods for genotypes as defined in the VCF specification">
##GATKCommandLine.HaplotypeCaller=<ID=HaplotypeCaller,Version=3.8-0-ge9d806836,Date="Sun Oct 29 01:03:03 UTC 2017",Epoch=1509238983333,CommandLineOptions="analysis_type=HaplotypeCaller input_file=[./ERR1977350.arrg.mrkdup.sorted.bam] showFullBamList=false read_buffer_size=null read_filter=[] disable_read_filter=[] intervals=null excludeInterv
als=null interval_set_rule=UNION interval_merging=ALL interval_padding=0 reference_sequence=reference/hg38.fa nonDeterministicRandomSeed=false disableDithering=false maxRuntime=-1 maxRuntimeUnits=MINUTES downsampling_type=BY_SAMPLE downsample_to_fraction=null downsample_to_coverage=500 baq=OFF baqGapOpenPenalty=40.0 refactor_NDN_cigar_string=
false fix_misencoded_quality_scores=false allow_potentially_misencoded_quality_scores=false useOriginalQualities=false defaultBaseQualities=-1 performanceLog=null BQSR=null quantize_quals=0 static_quantized_quals=null round_down_quantized=false disable_indel_quals=false emit_original_quals=false preserve_qscores_less_than=6 globalQScorePrior=
-1.0 secondsBetweenProgressUpdates=10 validation_strictness=SILENT remove_program_records=false keep_program_records=false sample_rename_mapping_file=null unsafe=null use_jdk_deflater=false use_jdk_inflater=false disable_auto_index_creation_and_locking_when_reading_rods=false no_cmdline_in_header=false sites_only=false never_trim_vcf_format_f
ield=false bcf=false bam_compression=null simplifyBAM=false disable_bam_indexing=false generate_md5=false num_threads=1 num_cpu_threads_per_data_thread=1 num_io_threads=0 monitorThreadEfficiency=false num_bam_file_handles=null read_group_black_list=null pedigree=[] pedigreeString=[] pedigreeValidationType=STRICT allow_intervals_with_unindexed
_bam=false generateShadowBCF=false variant_index_type=DYNAMIC_SEEK variant_index_parameter=-1 reference_window_stop=0 phone_home= gatk_key=null tag=NA logging_level=INFO log_to_file=null help=false version=false out=/data/project/./ERR1977350.arrg.mrkdup.sorted.bam.vcf likelihoodCalculationEngine=PairHMM heterogeneousKmerSizeResolution=COMBO_
MIN dbsnp=(RodBinding name= source=UNBOUND) dontTrimActiveRegions=false maxDiscARExtension=25 maxGGAARExtension=300 paddingAroundIndels=150 paddingAroundSNPs=20 comp=[] annotation=[] excludeAnnotation=[] group=[StandardAnnotation, StandardHCAnnotation] debug=false useFilteredReadsForAnnotations=false emitRefConfidence=NONE bamOutput=null bamW
riterType=CALLED_HAPLOTYPES emitDroppedReads=false disableOptimizations=false annotateNDA=false useNewAFCalculator=false heterozygosity=0.001 indel_heterozygosity=1.25E-4 heterozygosity_stdev=0.01 standard_min_confidence_threshold_for_calling=30.0 standard_min_confidence_threshold_for_emitting=30.0 max_alternate_alleles=6 max_genotype_count=1
024 max_num_PL_values=100 input_prior=[] sample_ploidy=2 genotyping_mode=DISCOVERY alleles=(RodBinding name= source=UNBOUND) contamination_fraction_to_filter=0.0 contamination_fraction_per_sample_file=null p_nonref_model=null exactcallslog=null output_mode=EMIT_VARIANTS_ONLY allSitePLs=false gcpHMM=10 pair_hmm_implementation=FASTEST_AVAILABLE
 phredScaledGlobalReadMismappingRate=45 noFpga=false nativePairHmmThreads=1 useDoublePrecision=false sample_name=null kmerSize=[10, 25] dontIncreaseKmerSizesForCycles=false allowNonUniqueKmersInRef=false numPruningSamples=1 recoverDanglingHeads=false doNotRecoverDanglingBranches=false minDanglingBranchLength=4 consensus=false maxNumHaplotypes
InPopulation=128 errorCorrectKmers=false minPruning=2 debugGraphTransformations=false allowCyclesInKmerGraphToGeneratePaths=false graphOutput=null kmerLengthForReadErrorCorrection=25 minObservationsForKmerToBeSolid=20 GVCFGQBands=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31
, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 70, 80, 90, 99] indelSizeToEliminateInRefModel=10 min_base_quality_score=10 includeUmappedReads=false useAllelesTrigger=false doNotRunPhysicalPhasing=true keepRG=null justDetermineActiveRegions=false dontGenotype=false dontUse
SoftClippedBases=false captureAssemblyFailureBAM=false errorCorrectReads=false pcr_indel_model=CONSERVATIVE maxReadsInRegionPerSample=10000 minReadsPerAlignmentStart=10 mergeVariantsViaLD=false activityProfileOut=null activeRegionOut=null activeRegionIn=null activeRegionExtension=null forceActive=false activeRegionMaxSize=null bandPassSigma=n
ull maxReadsInMemoryPerSample=30000 maxTotalReadsInMemory=10000000 maxProbPropagationDistance=50 activeProbabilityThreshold=0.002 min_mapping_quality_score=20 filter_reads_with_N_cigar=false filter_mismatching_base_and_quals=false filter_bases_not_stored=false">
##INFO=<ID=AC,Number=A,Type=Integer,Description="Allele count in genotypes, for each ALT allele, in the same order as listed">
##INFO=<ID=AF,Number=A,Type=Float,Description="Allele Frequency, for each ALT allele, in the same order as listed">
##INFO=<ID=AN,Number=1,Type=Integer,Description="Total number of alleles in called genotypes">
##INFO=<ID=BaseQRankSum,Number=1,Type=Float,Description="Z-score from Wilcoxon rank sum test of Alt Vs. Ref base qualities">
##INFO=<ID=ClippingRankSum,Number=1,Type=Float,Description="Z-score From Wilcoxon rank sum test of Alt vs. Ref number of hard clipped bases">
##INFO=<ID=DP,Number=1,Type=Integer,Description="Approximate read depth; some reads may have been filtered">
##INFO=<ID=DS,Number=0,Type=Flag,Description="Were any of the samples downsampled?">
##INFO=<ID=ExcessHet,Number=1,Type=Float,Description="Phred-scaled p-value for exact test of excess heterozygosity">
##INFO=<ID=FS,Number=1,Type=Float,Description="Phred-scaled p-value using Fisher's exact test to detect strand bias">
##INFO=<ID=HaplotypeScore,Number=1,Type=Float,Description="Consistency of the site with at most two segregating haplotypes">
##INFO=<ID=InbreedingCoeff,Number=1,Type=Float,Description="Inbreeding coefficient as estimated from the genotype likelihoods per-sample when compared against the Hardy-Weinberg expectation">
##INFO=<ID=MLEAC,Number=A,Type=Integer,Description="Maximum likelihood expectation (MLE) for the allele counts (not necessarily the same as the AC), for each ALT allele, in the same order as listed">
##INFO=<ID=MLEAF,Number=A,Type=Float,Description="Maximum likelihood expectation (MLE) for the allele frequency (not necessarily the same as the AF), for each ALT allele, in the same order as listed">
##INFO=<ID=MQ,Number=1,Type=Float,Description="RMS Mapping Quality">
##INFO=<ID=MQRankSum,Number=1,Type=Float,Description="Z-score From Wilcoxon rank sum test of Alt vs. Ref read mapping qualities">
##INFO=<ID=QD,Number=1,Type=Float,Description="Variant Confidence/Quality by Depth">
##INFO=<ID=ReadPosRankSum,Number=1,Type=Float,Description="Z-score from Wilcoxon rank sum test of Alt vs. Ref read position bias">
##INFO=<ID=SOR,Number=1,Type=Float,Description="Symmetric Odds Ratio of 2x2 contingency table to detect strand bias">
##contig=<ID=chr1,length=248956422>
##contig=<ID=chr10,length=133797422>
##contig=<ID=chr11,length=135086622>
##contig=<ID=chr11_KI270721v1_random,length=100316>
##contig=<ID=chr12,length=133275309>
##contig=<ID=chr13,length=114364328>
##contig=<ID=chr14,length=107043718>
##contig=<ID=chr14_GL000009v2_random,length=201709>
##contig=<ID=chr14_GL000225v1_random,length=211173>
##contig=<ID=chr14_KI270722v1_random,length=194050>
##contig=<ID=chr14_GL000194v1_random,length=191469>
##contig=<ID=chr14_KI270723v1_random,length=38115>
##contig=<ID=chr14_KI270724v1_random,length=39555>
##contig=<ID=chr14_KI270725v1_random,length=172810>
##contig=<ID=chr14_KI270726v1_random,length=43739>
##contig=<ID=chr15,length=101991189>
##contig=<ID=chr15_KI270727v1_random,length=448248>
##contig=<ID=chr16,length=90338345>
##contig=<ID=chr16_KI270728v1_random,length=1872759>
##contig=<ID=chr17,length=83257441>
##contig=<ID=chr17_GL000205v2_random,length=185591>
##contig=<ID=chr17_KI270729v1_random,length=280839>
##contig=<ID=chr17_KI270730v1_random,length=112551>
##contig=<ID=chr18,length=80373285>
##contig=<ID=chr19,length=58617616>
##contig=<ID=chr1_KI270706v1_random,length=175055>
##contig=<ID=chr1_KI270707v1_random,length=32032>
##contig=<ID=chr1_KI270708v1_random,length=127682>
##contig=<ID=chr1_KI270709v1_random,length=66860>
##contig=<ID=chr1_KI270710v1_random,length=40176>
##contig=<ID=chr1_KI270711v1_random,length=42210>
##contig=<ID=chr1_KI270712v1_random,length=176043>
##contig=<ID=chr1_KI270713v1_random,length=40745>

##reference=file:///data/project/reference/hg38.fa
#CHROM  POS     ID      REF     ALT     QUAL    FILTER  INFO    FORMAT  ERR1977350
chr1    14930   .       A       G       117.90  .       AC=2;AF=1.00;AN=2;DP=5;ExcessHet=3.0103;FS=0.000;MLEAC=2;MLEAF=1.00;MQ=24.43;QD=23.58;SOR=1.981 GT:AD:DP:GQ:PL  1/1:0,5:5:15:146,15,0
chr1    15118   .       A       G       33.77   .       AC=1;AF=0.500;AN=2;BaseQRankSum=-0.652;ClippingRankSum=0.000;DP=7;ExcessHet=3.0103;FS=3.680;MLEAC=1;MLEAF=0.500;MQ=23.93;MQRankSum=1.068;QD=4.82;ReadPosRankSum=-0.697;SOR=2.833        GT:AD:DP:GQ:PL  0/1:4,3:7:62:62,0,88
chr1    16378   .       T       C       31.77   .       AC=1;AF=0.500;AN=2;BaseQRankSum=-0.842;ClippingRankSum=0.000;DP=5;ExcessHet=3.0103;FS=3.979;MLEAC=1;MLEAF=0.500;MQ=20.62;MQRankSum=1.036;QD=6.35;ReadPosRankSum=-0.524;SOR=0.859        GT:AD:DP:GQ:PL  0/1:2,3:5:34:60,0,34
chr1    16495   .       G       C       165.77  .       AC=1;AF=0.500;AN=2;BaseQRankSum=0.496;ClippingRankSum=0.000;DP=19;ExcessHet=3.0103;FS=2.017;MLEAC=1;MLEAF=0.500;MQ=20.90;MQRankSum=-1.364;QD=8.72;ReadPosRankSum=0.047;SOR=0.243        GT:AD:DP:GQ:PL  0/1:9,10:19:99:194,0,178
chr1    16841   .       G       T       49.77   .       AC=1;AF=0.500;AN=2;BaseQRankSum=0.425;ClippingRankSum=0.000;DP=36;ExcessHet=3.0103;FS=8.105;MLEAC=1;MLEAF=0.500;MQ=22.32;MQRankSum=2.819;QD=1.38;ReadPosRankSum=-1.272;SOR=3.014        GT:AD:DP:GQ:PL  0/1:29,7:36:78:78,0,734
chr1    17385   .       G       A       94.77   .       AC=1;AF=0.500;AN=2;BaseQRankSum=-0.738;ClippingRankSum=0.000;DP=80;ExcessHet=3.0103;FS=4.409;MLEAC=1;MLEAF=0.500;MQ=36.51;MQRankSum=0.080;QD=1.18;ReadPosRankSum=-0.884;SOR=0.057       GT:AD:DP:GQ:PL  0/1:69,11:80:99:123,0,2171
chr1    133129  .       G       A       156.77  .       AC=1;AF=0.500;AN=2;BaseQRankSum=-0.631;ClippingRankSum=0.000;DP=16;ExcessHet=3.0103;FS=6.726;MLEAC=1;MLEAF=0.500;MQ=40.07;MQRankSum=-0.839;QD=9.80;ReadPosRankSum=-1.264;SOR=3.086      GT:AD:DP:GQ:PL  0/1:9,7:16:99:185,0,301
chr1    135040  .       T       C       374.77  .       AC=1;AF=0.500;AN=2;BaseQRankSum=0.065;ClippingRankSum=0.000;DP=61;ExcessHet=3.0103;FS=2.413;MLEAC=1;MLEAF=0.500;MQ=31.50;MQRankSum=-0.487;QD=6.35;ReadPosRankSum=-0.244;SOR=0.377       GT:AD:DP:GQ:PL  0/1:40,19:59:99:403,0,1099
chr1    135195  .       A       G       739.77  .       AC=1;AF=0.500;AN=2;BaseQRankSum=0.267;ClippingRankSum=0.000;DP=95;ExcessHet=3.0103;FS=2.972;MLEAC=1;MLEAF=0.500;MQ=48.27;MQRankSum=0.432;QD=7.87;ReadPosRankSum=0.570;SOR=0.542 GT:AD:DP:GQ:PL  0/1:65,29:94:99:768,0,1999
chr1    138156  .       G       T       233.77  .       AC=1;AF=0.500;AN=2;BaseQRankSum=-1.049;ClippingRankSum=0.000;DP=48;ExcessHet=3.0103;FS=3.028;MLEAC=1;MLEAF=0.500;MQ=22.03;MQRankSum=2.304;QD=4.87;ReadPosRankSum=1.002;SOR=1.355        GT:AD:DP:GQ:PL  0/1:33,15:48:99:262,0,715

### Excerpt of 'FORMAT' and Sample Data fields

In [ ]:
GT:AD:DP:GQ:PL  0/1:4,3:7:62:62,0,88
GT:AD:DP:GQ:PL  0/1:2,3:5:34:60,0,34     
GT:AD:DP:GQ:PL  0/1:9,10:19:99:194,0,178